In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Deep Learning Libraries
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model

In [ ]:
def accuracy_loss_plot(history):
    plt.Figure()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='lower right')
    plt.show()
    plt.Figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='lower right')
    plt.show()

In [ ]:
data = np.load('fashion_mnist_dataset_train.npy', allow_pickle=True).item()

In [ ]:
s = pd.Series(data['target'])
target = s.replace([1, 2, 3, 4, 5], [0, 1, 2, 3, 4])
target = target.to_numpy()
target

In [ ]:
from sklearn.model_selection import train_test_split
# Splitting the data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(data['features'], target, test_size=0.2, random_state=275)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=275)

In [ ]:
#pad the images to achieve 32 x 32
X_train = tf.pad(X_train, [[0, 0], [2,2], [2,2]])
X_val = tf.pad(X_val, [[0, 0], [2,2], [2,2]])
X_test = tf.pad(X_test, [[0, 0], [2,2], [2,2]])
#expand and repeat to create 3 channels
X_train = tf.expand_dims(X_train, axis=3, name=None)
X_val = tf.expand_dims(X_val, axis=3, name=None)
X_test = tf.expand_dims(X_test, axis=3, name=None)


In [ ]:
print(X_train.shape, '\n')
print(X_val.shape, '\n')
print(X_test.shape, '\n')

In [ ]:
print(tf.__version__)

In [ ]:
X_train = tf.repeat(X_train, 3, axis=3)
X_val = tf.repeat(X_train, 3, axis=3)
X_test = tf.repeat(X_test, 3, axis=3)

In [ ]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(5, activation = 'softmax')(x)

In [ ]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
history = head_model.fit(X_train, y_train, batch_size=64, epochs=40, validation_data=(X_val, y_val))